# Import libraries

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 

In [2]:
import pandas as pd  
import numpy as np
from statsmodels import robust
from pandas import read_excel
from scipy import spatial
from numpy.linalg import norm
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import mplcursors
import random
from matplotlib import gridspec
import tensorflow as tf
import os
from concurrent.futures import ProcessPoolExecutor
from numba import jit


2024-11-26 11:52:25.166860: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-26 11:52:26.061917: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# 1. Read the data


In [3]:
#file_name = "/home/gauss/Desktop/Pretrained baseline EfficientNetB0 Bbox Size 50/KB_Normalized/alldata.xlsx"
#file_name = "/home/gauss/Desktop/CellProfiler_Features/O8W_normalized.xlsx"
file_name = "/home/gauss/Desktop/Revised_Data_Features/Pretrained baseline MobileNetV2 Bbox Size 90/MHB_normalized/alldata.xlsx"

data = pd.read_excel(file_name)
data['Plate number'] = data['Plate number'].apply(str)
#data.head()

# 2. Correlation matrix 

## To calculate enrichment score there are two ways:
### 1. Calculate enrichment for each combination i.e KB, O3B, MHB etc 

In [4]:
#if correlation matrix across entire 8 plates for each combination is required
cat = ["MHB"]
data = data.loc[data['Combination'].isin(cat)]
#data

### 2. Compute correlation matrix across entire dataset and then compute Enrichment

In [5]:
new_data = data.drop(columns=[ "Unnamed: 0","Unnamed: 0.1","Image", "Label"], axis=1) #For neural network
#new_data = data.drop(columns=[ "Unnamed: 0","Unnamed: 0.1", "Label", "FileName_Cancer", "FileName_Cellpose", "FileName_Dapi", "FileName_Fibroblast", "ImageNumber"], axis=1) #for cellprofiler features

new_data['well_annotation'] =  new_data['Plate number']+ '-' + new_data['Combination']+ '-' + new_data['Well_number'] + '-' + new_data['Compound'] +  '-' + new_data['MoA'] 
new_data['well_num'] =  new_data['Plate number'] + '-' + new_data['Combination']+ '-' + new_data['Well_number']
data_new = new_data.drop(columns=["Plate number", "Combination","Concentration","Compound","Well_number", "MoA", "Category", "well_annotation", "well_num"], axis=1)

#prepare dataframe for correlation matrix
data_corr = new_data.drop(columns=["Plate number", "Combination", "Concentration","Compound", "MoA", "Category", "well_annotation", "Well_number"], axis=1)
#Put well_num col as first and set it as index col
first_column = data_corr.pop('well_num')
data_corr.insert(0, 'well_num', first_column)
data_corr.set_index('well_num', inplace=True)

#transpose the dataframe and compute coorelation coefficient
data_new_T = data_corr.T
correlation_mat = data_new_T.corr()
correlation_mat.head()

well_num,1-MHB-A1,1-MHB-A2,1-MHB-A3,1-MHB-A4,1-MHB-A5,1-MHB-A6,1-MHB-A7,1-MHB-A8,1-MHB-A9,1-MHB-A10,...,8-MHB-P15,8-MHB-P16,8-MHB-P17,8-MHB-P18,8-MHB-P19,8-MHB-P20,8-MHB-P21,8-MHB-P22,8-MHB-P23,8-MHB-P24
well_num,,,,,,,,,,,,,,,,,,,,,
1-MHB-A1,1.000000,0.507496,0.166784,0.391302,0.567140,0.649643,0.552382,0.456484,0.477330,0.353297,...,0.485046,0.159148,0.407839,0.357393,0.753098,0.556387,0.717361,0.707414,0.307666,NaN
1-MHB-A2,0.507496,1.000000,0.510651,0.081219,0.239589,0.390522,0.914682,0.907079,0.434345,0.763072,...,0.238504,0.267383,0.795990,0.691138,0.497880,0.408364,0.393692,0.405805,0.757733,NaN
1-MHB-A3,0.166784,0.510651,1.000000,-0.125245,0.110656,0.150146,0.695885,0.567263,0.446356,0.670658,...,0.065684,0.438507,0.446300,0.444365,0.105955,0.068750,0.084872,0.041609,0.494415,NaN
1-MHB-A4,0.391302,0.081219,-0.125245,1.000000,0.635695,0.572061,-0.019963,0.061140,0.268130,0.078954,...,0.575515,-0.176154,-0.025557,0.156542,0.490531,0.538845,0.565420,0.624349,0.021563,NaN
1-MHB-A5,0.567140,0.239589,0.110656,0.635695,1.000000,0.620917,0.205842,0.170997,0.124816,0.058234,...,0.319083,-0.068354,0.195090,0.257074,0.495594,0.642848,0.509863,0.491265,0.118303,NaN


In [6]:
X = data_new.iloc[:,:].values #features on which distance will be calculated
y = new_data["well_annotation"] #well annotation to map the distance index later
z = new_data["Category"]
conc = data["Concentration"] #change to "data" in case using full dataframe
y = y.values
z = z.values
conc = conc.values
'''
print(X)
print(y)
print(z)
print(conc)
'''

'\nprint(X)\nprint(y)\nprint(z)\nprint(conc)\n'

# 4. Calculating enrichment score and Pvalue after excluding query compound

In [7]:
well_annot = []
well_conc = []
well_cat = []
highest_es = []
pval = []

@jit(nopython=True)
def compute_res(correlation_matrix, well_category, cat_moa):
    M = np.abs(correlation_matrix[well_category == cat_moa]).sum()
    T = (well_category != cat_moa).sum()
    res = np.zeros(len(correlation_matrix))
    res[0] = correlation_matrix[0] * (1/M)
    for j in range(1, len(correlation_matrix)):
        if well_category[j] == cat_moa:
            res[j] = res[j-1] + (correlation_matrix[j] * (1 / M))
        else:
            res[j] = res[j-1] - (1 / T)
    return res


def process_category(category, new_data, y, z, conc, correlation_mat, cat_moa):
    # Get the compound name and concentration for the current category
    current_compound = new_data.loc[category, 'Compound']
    current_concentration = new_data.loc[category, 'Concentration']
    
    #print("current compound which will be excluded:", current_compound)
    #print("current concentration :", current_concentration)

    # Create a mask to keep all data except the other concentrations of the current compound
    mask = ~((new_data['Compound'] == current_compound) & (new_data['Concentration'] != current_concentration))
    
    # Apply the mask to create filtered data
    filtered_new_data = new_data[mask].reset_index(drop=True)
    filtered_y = y[mask]
    filtered_z = z[mask]
    filtered_conc = conc[mask]
    
    # Create the correlation matrix for the filtered data
    filtered_correlation_mat = correlation_mat.loc[filtered_new_data['well_num'], filtered_new_data['well_num']]
    
    # Get the correlation values for the current category
    current_correlations = filtered_correlation_mat[new_data.loc[category, 'well_num']]
    
    data_enrch = pd.DataFrame({
        'Well_annotation': filtered_y,
        'Well_category': filtered_z,
        'Well_concentration': filtered_conc,
        'Correlation_matrix': current_correlations
    })
    data_enrch = data_enrch.sort_values(by='Correlation_matrix', ascending=False).reset_index(drop=True)
    
    correlation_matrix = data_enrch['Correlation_matrix'].values
    well_category = (data_enrch['Well_category'] == cat_moa).values
    
    res = compute_res(correlation_matrix, well_category, True)
    res_original = res.max()
    #print(res_original)
    res_max = []
    for _ in range(1000):
        np.random.shuffle(well_category[1:])
        res = compute_res(correlation_matrix, well_category, True)
        res_max.append(res.max())
    count = np.sum(np.array(res_max) >= res_original)    
    p_val = count/1000
    
    return {
        'Well_annotation': new_data['well_annotation'].loc[category],
        'Well_concentration': new_data['Concentration'].loc[category],
        'Well_Category': cat_moa,
        'Highest_ES': res_original,
        'Pvalue': p_val
    }
def main():
    moa = ['EGFR', 'VEGFR', 'PI3K', 'Topoisomerase', 'Mitotic', 'MEK1/2', 'PARP', 'CDK', 'BET', 'HDAC']    
    results = []
    with ProcessPoolExecutor() as executor:
        for m in moa:
            print(m)
            categories = new_data.loc[new_data['Category'] == m].index.tolist()
            futures = [executor.submit(process_category, category, new_data, y, z, conc, correlation_mat, m) for category in categories]
            results.extend([future.result() for future in futures])
    data_Pval = pd.DataFrame(results)
    return data_Pval
if __name__ == '__main__':
    data_Pval = main()

EGFR
VEGFR
PI3K
Topoisomerase
Mitotic
MEK1/2
PARP
CDK
BET
HDAC


In [8]:
data_Pval_enriched = data_Pval[data_Pval['Pvalue'] < 0.05]
data_Pval_enriched = data_Pval_enriched['Well_Category'].value_counts().rename_axis('MoA').reset_index(name='value')
data_Pval_enriched

,MoA,value
0,HDAC,67
1,EGFR,61
2,PI3K,48
3,Topoisomerase,40
4,VEGFR,38
5,Mitotic,30
6,BET,26
7,MEK1/2,25
8,PARP,19
9,CDK,10


In [9]:
data_Pval_enriched.to_excel("/home/gauss/Desktop/Revised_Data_Features/Pretrained baseline MobileNetV2 Bbox Size 90/MHB_normalized/pval_norm_exclquerycomp_MHB.xlsx")


data_Pval.to_excel("/home/gauss/Desktop/Revised_Data_Features/Pretrained baseline MobileNetV2 Bbox Size 90/MHB_normalized/All_MHB_ES_Pval_exclquerycomp.xlsx")


#count compounds with 1st and 2nd lowest concentration which are enriched

# Extract the drug name
data_Pval['Drug'] = data_Pval['Well_annotation'].apply(lambda x: x.split('-')[-1])

# Sort by Drug and Well_concentration, then filter for the lowest two concentrations with Pvalue < 0.05
filtered_data = (
    data_Pval
    .sort_values(['Drug', 'Well_concentration'])  # Sort by drug and concentration
    .groupby('Drug')                              # Group by drug
    .head(2)                                      # Take the lowest 2 concentrations per drug
)

# Count the entries with Pvalue < 0.05 in these filtered rows
count_low_pvalue = (filtered_data['Pvalue'] < 0.05).sum()

print(f"Number of drugs with Pvalue < 0.05 in the lowest 2 concentrations: {count_low_pvalue}")
filtered_data.to_excel("/home/gauss/Desktop/Revised_Data_Features/Pretrained baseline MobileNetV2 Bbox Size 90/MHB_normalized/lowest2conc_comp_es_exclquerycomp_MHB.xlsx")


Number of drugs with Pvalue < 0.05 in the lowest 2 concentrations: 69
